# 初期設定

In [1]:
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests #HTTP操作用
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

/Users/yutomatsuda/Lecture/DSProg2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')
wait = WebDriverWait(options,10)

# スクレイピング

In [3]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://www.google.co.jp/"
driver.get(url)
time.sleep(3)
print("サイトのタイトル：", driver.title)

サイトのタイトル： Google


In [4]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)
for a in range(1,21):
    house_name=driver.find_element(By.XPATH,f'//*[@id="tr_{a}"]/td[4]/div/div/span/a').text
    print(house_name)
    house_link_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
    house_link = house_link_element.get_attribute("href")
    print(house_link)

ウインマリリン
https://db.netkeiba.com/horse/2017101431
ディープボンド
https://db.netkeiba.com/horse/2017102170
ホウオウエミーズ
https://db.netkeiba.com/horse/2017104691
ヒートオンビート
https://db.netkeiba.com/horse/2017104936
ディアスティマ
https://db.netkeiba.com/horse/2017105379
アイアンバローズ
https://db.netkeiba.com/horse/2017105477
タイトルホルダー
https://db.netkeiba.com/horse/2018103559
シャフリヤール
https://db.netkeiba.com/horse/2018105165
スルーセブンシーズ
https://db.netkeiba.com/horse/2018105269
プラダリア
https://db.netkeiba.com/horse/2019100109
マテンロウレオ
https://db.netkeiba.com/horse/2019102879
ライラック
https://db.netkeiba.com/horse/2019103588
スターズオンアース
https://db.netkeiba.com/horse/2019104740
ドウデュース
https://db.netkeiba.com/horse/2019105283
ジャスティンパレス
https://db.netkeiba.com/horse/2019105346
ブローザホーン
https://db.netkeiba.com/horse/2019105748
ソールオリエンス
https://db.netkeiba.com/horse/2020102899
ハーパー
https://db.netkeiba.com/horse/2020103458
タスティエーラ
https://db.netkeiba.com/horse/2020103532
ドゥラエレーデ
https://db.netkeiba.com/horse/2020103626


In [16]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)

url = "https://db.netkeiba.com/horse/2017101431"
driver.get(url)
time.sleep(0.1)

winscore_list = []
wintime_list = []
last_list = []
weight_change_list = []

b = 1
while True:
    try:
        # 勝率の取得
        winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
        if winscore and winscore.isdigit():  # 数字のみを確認
            winscore_list.append(float(winscore))

        # 着差の取得（空白や**を回避）
        wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
        if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
            wintime_list.append(float(wintime))

        # 上りの取得（空白や**を回避）
        last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
        if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
            last_list.append(float(last))

        # 体重の増減の取得（空白を回避）
        weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
        match = re.search(r'\((\+|-)?(\d+)\)', weight)
        if match:  # マッチするものがあった場合
            weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
            weight_change_list.append(weight_change)

        b += 1

    except NoSuchElementException:
        # 要素が見つからない場合はループを終了
        break

# 数値データの平均を計算して出力
print("勝率平均:", np.mean(winscore_list) if winscore_list else "データなし")
print("着差平均:", np.mean(wintime_list) if wintime_list else "データなし")
print("上り平均:", np.mean(last_list) if last_list else "データなし")
print("馬体重増減平均:", np.mean(weight_change_list) if weight_change_list else "データなし")

driver.quit()

勝率平均: 5.476190476190476
着差平均: 1.05
上り平均: 36.472222222222214
馬体重増減平均: 1.0


In [4]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://db.netkeiba.com/horse/2017105477"
driver.get(url)
time.sleep(0.1)

father_link_list = []
f_grandfather_link_list = []
f_grandmother_link_list = []
mother_link_link = []
m_grandfather_link_list = []
m_grandmother_link_list = []

father = driver.find_element(By.XPATH, '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[1]/a') # 父
f_grandfather = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[2]/a') # 父→父
f_grandmother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[2]/td/a') # 父→母
mother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[1]/a') # 母
m_grandfather = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[2]/a') # 母→父
m_grandmother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[4]/td/a') # 母→母

house_link_father = father.get_attribute("href")
house_link_father = house_link_father.replace("ped","result")
house_link_f_grandfather = f_grandfather.get_attribute("href")
house_link_f_grandfather = house_link_f_grandfather.replace("ped","result")
house_link_f_grandmother = f_grandmother.get_attribute("href")
house_link_f_grandmother = house_link_f_grandmother.replace("ped","result")
house_link_mother = mother.get_attribute("href")
house_link_mother = house_link_mother.replace("ped","result")
house_link_m_grandfather = m_grandfather.get_attribute("href")
house_link_m_grandfather = house_link_m_grandfather.replace("ped","result")
house_link_m_grandmother = m_grandmother.get_attribute("href")
house_link_m_grandmother = house_link_m_grandmother.replace("ped","result")

father_link_list.append(house_link_father)
f_grandfather_link_list.append(house_link_f_grandfather)
f_grandmother_link_list.append(house_link_f_grandmother)
mother_link_link.append(house_link_mother)
m_grandfather_link_list.append(house_link_m_grandfather)
m_grandmother_link_list.append(house_link_m_grandmother)

https://db.netkeiba.com/horse/result/2008102636/


In [6]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://db.netkeiba.com/horse/result/2008102636/"
driver.get(url)
time.sleep(0.1)
winsocore_list_1 = []

c = 1
while True:
    try:
        winsocore_1 = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[2]/div[1]/table/tbody/tr[{c}]/td[12]').text
        winsocore_1 = float(winsocore_1)
        winsocore_list_1.append(float(winsocore_1))
        c += 1

    except NoSuchElementException:
        # 要素が見つからない場合はループを終了
        break

# 取得した数値データの平均を計算
average_value = np.mean(winsocore_list_1)
print("平均値:", average_value)

平均値: 2.2857142857142856


# 統合

In [22]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)
houselink_list = []
datum = {}
for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name=driver.find_element(By.XPATH,f'//*[@id="tr_{a}"]/td[4]/div/div/span/a').text
        print(house_name)
        house_link_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        datum['馬名'] = house_name
        house_link = house_link_element.get_attribute("href")
        houselink_list.append(house_link)

    except NoSuchElementException:
        # 要素が見つからなかった場合は、スキップして次の要素へ
        continue
    datum['馬名'] = house_name

for houselink in houselink_list:
    driver = webdriver.Chrome()
    #指定したドライバーが見つかるまで待機
    driver.implicitly_wait(10)
    driver.get(houselink)
    time.sleep(0.1)
    winscore_list = []
    wintime_list = []
    last_list = []
    weight_list = []

    b = 1
    while True:
        try:
            # 勝率の取得
            winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
            if winscore and winscore.isdigit():  # 数字のみを確認
                winscore_list.append(float(winscore))

            # 着差の取得（空白や**を回避）
            wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
            if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                wintime_list.append(float(wintime))

            # 上りの取得（空白や**を回避）
            last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
            if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                last_list.append(float(last))

            # 体重の増減の取得（空白を回避）
            weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
            match = re.search(r'\((\+|-)?(\d+)\)', weight)
            if match:  # マッチするものがあった場合
                weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
                weight_change_list.append(weight_change)

            b += 1

        except NoSuchElementException:
            # 要素が見つからない場合はループを終了
            break


    # 取得した数値データの平均を計算
    winscore_value = np.mean(winscore_list)
    wintime_value = np.mean(wintime_list)
    last_value = np.mean(last_list)
    weight_value = np.mean(weight_list)
    print(wintime_value)

    datum['勝率平均'] = winscore_value
    datum['着差平均'] = wintime_value
    datum['上り平均'] = last_value
    datum['馬体重増減平均'] = weight_value 

    father_link_list = []
    f_grandfather_link_list = []
    f_grandmother_link_list = []
    mother_link_link = []
    m_grandfather_link_list = []
    m_grandmother_link_list = []

    father = driver.find_element(By.XPATH, '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[1]/a') # 父
    f_grandfather = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[2]/a') # 父→父
    f_grandmother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[2]/td/a') # 父→母
    mother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[1]/a') # 母
    m_grandfather = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[2]/a') # 母→父
    m_grandmother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[4]/td/a') # 母→母

    house_link_father = father.get_attribute("href")
    house_link_father = house_link_father.replace("ped","result")
    house_link_f_grandfather = f_grandfather.get_attribute("href")
    house_link_f_grandfather = house_link_f_grandfather.replace("ped","result")
    house_link_f_grandmother = f_grandmother.get_attribute("href")
    house_link_f_grandmother = house_link_f_grandmother.replace("ped","result")
    house_link_mother = mother.get_attribute("href")
    house_link_mother = house_link_mother.replace("ped","result")
    house_link_m_grandfather = m_grandfather.get_attribute("href")
    house_link_m_grandfather = house_link_m_grandfather.replace("ped","result")
    house_link_m_grandmother = m_grandmother.get_attribute("href")
    house_link_m_grandmother = house_link_m_grandmother.replace("ped","result")

    father_link_list.append(house_link_father)
    f_grandfather_link_list.append(house_link_f_grandfather)
    f_grandmother_link_list.append(house_link_f_grandmother)
    mother_link_link.append(house_link_mother)
    m_grandfather_link_list.append(house_link_m_grandfather)
    m_grandmother_link_list.append(house_link_m_grandmother)

for father_link in father_link_list:
    driver = webdriver.Chrome()
    #指定したドライバーが見つかるまで待機
    driver.implicitly_wait(10)
    driver.get(father_link)
    time.sleep(0.1)
    winsocore_list_1 = []

    c = 1
    while True:
        try:
            winsocore_1 = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[2]/div[1]/table/tbody/tr[{c}]/td[12]').text
            winsocore_1 = float(winsocore_1)
            winsocore_list_1.append(float(winsocore_1))
            c += 1

        except NoSuchElementException:
            # 要素が見つからない場合はループを終了
            break

    # 取得した数値データの平均を計算
    average_value = np.mean(winsocore_list_1)
print(datum)

ウインマリリン
ディープボンド
ホウオウエミーズ
ヒートオンビート
アイアンバローズ
タイトルホルダー
シャフリヤール
スルーセブンシーズ
プラダリア
ライラック
スターズオンアース
ドウデュース
ジャスティンパレス
ソールオリエンス
ハーパー
タスティエーラ


/Users/yutomatsuda/Lecture/DSProg2/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/yutomatsuda/Lecture/DSProg2/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1.05
0.6736842105263158


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.109)
Stacktrace:
0   chromedriver                        0x00000001049b84dc chromedriver + 4162780
1   chromedriver                        0x00000001049b0664 chromedriver + 4130404
2   chromedriver                        0x0000000104607bc0 chromedriver + 293824
3   chromedriver                        0x00000001045e06c0 chromedriver + 132800
4   chromedriver                        0x000000010467925c chromedriver + 758364
5   chromedriver                        0x000000010468d294 chromedriver + 840340
6   chromedriver                        0x00000001046416bc chromedriver + 530108
7   chromedriver                        0x0000000104642930 chromedriver + 534832
8   chromedriver                        0x000000010497de08 chromedriver + 3923464
9   chromedriver                        0x00000001049823dc chromedriver + 3941340
10  chromedriver  

In [24]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import numpy as np
import time
import re

# WebDriverのインスタンスを一度だけ生成
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# URLを開く
url = "https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)

# 全ての馬のデータを格納するリスト
all_data = []

# 馬のリンクを取得
houselink_list = []
for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        house_name = house_name_element.text
        house_link = house_name_element.get_attribute("href")
        houselink_list.append(house_link)
        print(house_name)
    except NoSuchElementException:
        # 要素が見つからなかった場合は、スキップして次の要素へ
        continue

for houselink in houselink_list:
    driver = webdriver.Chrome()
    #指定したドライバーが見つかるまで待機
    driver.implicitly_wait(10)
    driver.get(houselink)
    time.sleep(0.1)
    winscore_list = []
    wintime_list = []
    last_list = []
    weight_list = []

    b = 1
    while True:
        try:
            # 勝率の取得
            winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
            if winscore and winscore.isdigit():  # 数字のみを確認
                winscore_list.append(float(winscore))

            # 着差の取得（空白や**を回避）
            wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
            if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                wintime_list.append(float(wintime))

            # 上りの取得（空白や**を回避）
            last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
            if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                last_list.append(float(last))

            # 体重の増減の取得（空白を回避）
            weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
            match = re.search(r'\((\+|-)?(\d+)\)', weight)
            if match:  # マッチするものがあった場合
                weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
                weight_change_list.append(weight_change)

            b += 1

        except NoSuchElementException:
            # 要素が見つからない場合はループを終了
            break

    # 各データの平均値を計算して辞書に追加
    datum['勝率平均'] = np.mean(winscore_list) if winscore_list else None
    datum['着差平均'] = np.mean(wintime_list) if wintime_list else None
    datum['上り平均'] = np.mean(last_list) if last_list else None
    datum['馬体重増減平均'] = np.mean(weight_change_list) if weight_change_list else None

    # 個々の馬のデータ辞書をリストに追加
    all_data.append(datum)

# 全てのデータを出力
for horse_data in all_data:
    print(horse_data)

# ドライバーを閉じる
driver.quit()


ウインマリリン
ディープボンド
ホウオウエミーズ
ヒートオンビート
アイアンバローズ
タイトルホルダー
シャフリヤール
スルーセブンシーズ
プラダリア
ライラック
スターズオンアース
ドウデュース
ジャスティンパレス
ソールオリエンス
ハーパー
タスティエーラ
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.8333333333333333, '着差平均': 0.25, '上り平均': 34.46666666666667, '馬体重増減平均': 0.6413502109704642}
{'馬名': 'タスティエーラ', '勝率平均': 1.833333333333333